# Simulate random walks with technical indicators

In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from functools import partial

from quantfinlib.sim import BrownianMotion, QuasiBrownianMotion
from quantfinlib.feature.indicators import rsi, ewm_rsi, macd, macd_signal, ewm_mom
from quantfinlib.feature.indicators import EwmBollingerBands, DonchianBands, GenericBands

In [ ]:
def rsi_based_signal(prices: np.ndarray, func=rsi) -> np.ndarray:
    sig = func(prices)
    sig = np.where((sig > 30) & (sig < 70), 50, sig)
    return (50 - sig) / 100

def bands_based_signal(prices: np.ndarray, bands: GenericBands) -> np.ndarray:
    sig = bands(prices).percent_b()
    sig = np.where((sig > 0.3) & (sig < 0.7), 0.5, sig)
    return 0.5 - sig

def calculate_predictive_corr(prices: np.ndarray, indicator: np.ndarray) -> float:
    returns = np.diff(prices)
    return np.corrcoef(np.nan_to_num(indicator[:-1]), returns)[0, 1]

## Settings

In [ ]:
params = {'drift': 0.5, 'vol': 0.30}
path_params = {
    'x0': 100,
    'dt': 1/252,
    'num_steps': 2520,
    'num_paths': 1,
    'label_start': '2020-01-01',
    'label_freq': 'B',
    'random_state': 42
}
price_color = px.colors.qualitative.Plotly[0]

# True Brownian Motion for comparison
model = BrownianMotion(**params)
p_bm = model.path_sample(**path_params)

# Brownian motion adjusted with Relative Strength Index
The Relative Strength Index (RSI) is a momentum oscillator that measures the speed and change of price movements. It is used to identify overbought or oversold conditions in a market. The RSI ranges from 0 to 100. An asset is considered overbought when the RSI is above 70 and oversold when the RSI is below 30.

In this example we compare true Brownian motion with Brownian motion adjusted with the RSI. We will add an RSI based signal scaled to have a volatility of 10% of the original Brownian motion volatility.

## Standard RSI

In [ ]:
qbm_model = QuasiBrownianMotion(rsi_based_signal, **params)
p_qbm = qbm_model.path_sample(**path_params)

fig = go.Figure()
fig.add_trace(go.Scatter(x=p_bm.index, y=p_bm, mode='lines', name='BM Price', yaxis='y2'))
fig.add_trace(go.Scatter(x=p_qbm.index, y=p_qbm, mode='lines', name='QBM Price', yaxis='y2',
                         line=dict(color=price_color, dash='dot')))
fig.add_trace(go.Scatter(x=p_bm.index, y=rsi(p_bm), mode='lines', name='RSI', yaxis='y'))
fig.update_layout(
    title='Brownian motion adjusted with Relative Strength Index', xaxis_title='Date', height=600,
    yaxis=dict(title='RSI', side='left', showgrid=False),
    yaxis2=dict(title='Closing Price', side='right', overlaying='y', showgrid=False)
)
fig.add_shape(type='line', x0=p_bm.index[0], x1=p_bm.index[-1], y0=30, y1=30, line=dict(color='black', width=1, dash='dash'))
fig.add_shape(type='line', x0=p_bm.index[0], x1=p_bm.index[-1], y0=70, y1=70, line=dict(color='black', width=1, dash='dash'))
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=p_qbm.index, y=p_qbm - p_bm, mode='lines', name='Diff Price',
                         line=dict(color="black", dash='dot')))
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Price Difference',
    height=200,
    margin=dict(r=150, t=10),
)
fig.show()

In [ ]:
print("Brownian motion:      ", calculate_predictive_corr(p_bm, rsi_based_signal(p_bm)))
print("Brownian motion + RSI:", calculate_predictive_corr(p_qbm, rsi_based_signal(p_qbm)))

## Exponentially weighted RSI

In [ ]:
ewmrsi_based_signal = partial(rsi_based_signal, func=ewm_rsi)
eqbm_model = QuasiBrownianMotion(ewmrsi_based_signal, f_signal_vol=0.1, **params)
p_eqbm = eqbm_model.path_sample(**path_params)

fig = go.Figure()
fig.add_trace(go.Scatter(x=p_bm.index, y=p_bm, mode='lines', name='BM Price', yaxis='y2'))
fig.add_trace(go.Scatter(x=p_eqbm.index, y=p_eqbm, mode='lines', name='QBM Price', yaxis='y2',
                         line=dict(color=price_color, dash='dot')))
fig.add_trace(go.Scatter(x=p_bm.index, y=ewm_rsi(p_bm), mode='lines', name='EWMRSI', yaxis='y'))
fig.update_layout(
    title='Brownian motion adjusted with EWM Relative Strength Index', xaxis_title='Date', height=600,
    yaxis=dict(title='RSI', side='left', showgrid=False),
    yaxis2=dict(title='Closing Price', side='right', overlaying='y', showgrid=False)
)
fig.add_shape(type='line', x0=p_bm.index[0], x1=p_bm.index[-1], y0=30, y1=30, line=dict(color='black', width=1, dash='dash'))
fig.add_shape(type='line', x0=p_bm.index[0], x1=p_bm.index[-1], y0=70, y1=70, line=dict(color='black', width=1, dash='dash'))
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=p_eqbm.index, y=p_eqbm - p_bm, mode='lines', name='Diff Price',
                         line=dict(color="black", dash='dot')))
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Price Difference',
    height=200,
    margin=dict(r=150, t=10),
)
fig.show()

In [ ]:
print("Brownian motion:         ", calculate_predictive_corr(p_bm, ewmrsi_based_signal(p_bm)))
print("Brownian motion + EWMRSI:", calculate_predictive_corr(p_eqbm, ewmrsi_based_signal(p_eqbm)))

# Brownian motion adjusted with Moving Average Convergence Divergence

In [ ]:
macd_model = QuasiBrownianMotion(macd_signal, f_signal_vol=0.1, **params)
p_macd = macd_model.path_sample(**path_params)
macd_line = macd(p_bm)
signal = macd_signal(p_bm)

fig = go.Figure()
fig.add_trace(go.Scatter(x=p_bm.index, y=p_bm, mode='lines', name='BM Price', yaxis='y2'))
fig.add_trace(go.Scatter(x=p_macd.index, y=p_macd, mode='lines', name='QBM Price', yaxis='y2',
                         line=dict(color=price_color, dash='dot')))
fig.add_trace(go.Scatter(x=p_bm.index, y=macd_line, mode='lines', name='MACD', yaxis='y'))
fig.add_trace(go.Scatter(x=p_bm.index, y=macd_line - signal, mode='lines', name='9D EMA', yaxis='y'))
fig.update_layout(
    title='Brownian motion adjusted with Moving Average Convergence Divergence signal', xaxis_title='Date', height=600,
    yaxis=dict(title='MACD signal', side='left', showgrid=False),
    yaxis2=dict(title='Closing Price', side='right', overlaying='y', showgrid=False)
)
fig.show()

colors = np.where(signal < 0, 'red', 'green')
fig = go.Figure()
fig.add_trace(go.Bar(x=p_bm.index, y=signal, marker_color=colors, name='MACD signal', yaxis='y2'))
fig.add_trace(go.Scatter(x=p_macd.index, y=p_macd - p_bm, mode='lines', name='Diff Price',
                         line=dict(color="black", dash='dot'), yaxis='y'))
fig.update_layout(
    xaxis_title='Date',
    height=200,
    yaxis=dict(title='Price Difference', side='left', showgrid=False),
    yaxis2=dict(side='right', overlaying='y', showgrid=False),
    margin=dict(r=150, t=10),
)
fig.show()

In [ ]:
print("Brownian motion:       ", calculate_predictive_corr(p_bm, macd_signal(p_bm)))
print("Brownian + MACD signal:", calculate_predictive_corr(p_macd, macd_signal(p_macd)))

# Brownian motion adjusted with EWMA Momentum (128 days)

In [ ]:
semi_ewm_mom = partial(ewm_mom, span=128, min_periods=32)
ewma_model = QuasiBrownianMotion(semi_ewm_mom, f_signal_vol=0.1, **params)
p_ewma = ewma_model.path_sample(**path_params)
signal = semi_ewm_mom(p_bm)
colors = np.where(signal < 0, 'red', 'green')

fig = go.Figure()
fig.add_trace(go.Bar(x=p_bm.index, y=signal, marker_color=colors, opacity=0.5, name='EWM', yaxis='y'))
fig.add_trace(go.Scatter(x=p_bm.index, y=p_bm, mode='lines', name='BM Price', yaxis='y2',
                         line=dict(color=price_color)))
fig.add_trace(go.Scatter(x=p_ewma.index, y=p_ewma, mode='lines', name='QBM Price', yaxis='y2',
                         line=dict(color=price_color, dash='dot')))
fig.update_layout(
    title='Brownian motion adjusted with EWMA Momentum', xaxis_title='Date', height=600,
    yaxis=dict(title='Momentum signal', side='left', showgrid=False),
    yaxis2=dict(title='Closing Price', side='right', overlaying='y', showgrid=False)
)
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=p_ewma.index, y=p_ewma - p_bm, mode='lines', name='Diff Price',
                         line=dict(color="black", dash='dot'), yaxis='y'))
fig.update_layout(
    xaxis_title='Date',
    height=200,
    yaxis=dict(title='Price Difference', side='left', showgrid=False),
    yaxis2=dict(side='right', overlaying='y', showgrid=False),
    margin=dict(r=150, t=10),
)
fig.show()

In [ ]:
print("Brownian motion:       ", calculate_predictive_corr(p_bm, semi_ewm_mom(p_bm)))
print("Brownian + MACD signal:", calculate_predictive_corr(p_ewma, semi_ewm_mom(p_ewma)))

# Brownian motion adjusted with Bollinger Bands %B

In [ ]:
eb_based_signal = partial(bands_based_signal, bands=EwmBollingerBands)
eb_model = QuasiBrownianMotion(eb_based_signal, f_signal_vol=0.1, **params)
p_ebbm = eb_model.path_sample(**path_params)
signal = eb_based_signal(p_bm)
eb = EwmBollingerBands(p_bm)
colors = np.where(signal < 0, 'red', 'green')

fig = go.Figure()
fig.add_trace(go.Bar(x=p_bm.index, y=signal, marker_color=colors, opacity=0.5, name='Indicator', yaxis='y'))
fig.add_trace(go.Scatter(x=p_bm.index, y=p_bm, mode='lines', name='BM Price', yaxis='y2',
                         line=dict(color=price_color)))
fig.add_trace(go.Scatter(x=p_ebbm.index, y=p_ebbm, mode='lines', name='QBM Price', yaxis='y2',
                         line=dict(color=price_color, dash='dot')))
fig.add_trace(go.Scatter(x=p_bm.index, y=eb.lower(), mode='lines', name='Lower Bollinger Band', yaxis='y2'))
fig.add_trace(go.Scatter(x=p_bm.index, y=eb.upper(), fill='tonexty', fillcolor="rgba(200,200,0,0.2)", name='Upper Bollinger Band', yaxis='y2'))
fig.update_layout(
    title='Brownian motion adjusted with EWM Bollinger %B (interpreted as overbought/oversold signal)', xaxis_title='Date', height=600,
    yaxis=dict(title='Indicator', side='left', showgrid=False),
    yaxis2=dict(title='Closing Price', side='right', overlaying='y', showgrid=False)
)
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=p_ebbm.index, y=p_ebbm - p_bm, mode='lines', name='Diff Price',
                         line=dict(color="black", dash='dot'), yaxis='y'))
fig.update_layout(
    xaxis_title='Date',
    height=200,
    yaxis=dict(title='Price Difference', side='left', showgrid=False),
    yaxis2=dict(side='right', overlaying='y', showgrid=False),
    margin=dict(r=150, t=10),
)
fig.show()

In [ ]:
print("Brownian motion: ", calculate_predictive_corr(p_bm, eb_based_signal(p_bm)))
print("Brownian + BB %B:", calculate_predictive_corr(p_ebbm, eb_based_signal(p_ebbm)))

# Brownian motion adjusted with Donchian Bands %B

In [ ]:
db_based_signal = partial(bands_based_signal, bands=DonchianBands)

db_model = QuasiBrownianMotion(db_based_signal, f_signal_vol=0.1, **params)
p_dbbm = db_model.path_sample(**path_params)
signal = db_based_signal(p_bm)
db = DonchianBands(p_bm)
colors = np.where(signal < 0, 'red', 'green')

fig = go.Figure()
fig.add_trace(go.Bar(x=p_bm.index, y=signal, marker_color=colors, opacity=0.5, name='Indicator', yaxis='y'))
fig.add_trace(go.Scatter(x=p_bm.index, y=p_bm, mode='lines', name='BM Price', yaxis='y2',
                         line=dict(color=price_color)))
fig.add_trace(go.Scatter(x=p_dbbm.index, y=p_dbbm, mode='lines', name='QBM Price', yaxis='y2',
                         line=dict(color=price_color, dash='dot')))
fig.add_trace(go.Scatter(x=p_bm.index, y=db.lower(), mode='lines', name='Lower Donchian Band', yaxis='y2'))
fig.add_trace(go.Scatter(x=p_bm.index, y=db.upper(), fill='tonexty', fillcolor="rgba(200,200,0,0.2)", name='Upper Donchian Band', yaxis='y2'))
fig.update_layout(
    title='Brownian motion adjusted with Donchian %B (interpreted as overbought/oversold signal)', xaxis_title='Date', height=600,
    yaxis=dict(title='Indicator', side='left', showgrid=False),
    yaxis2=dict(title='Closing Price', side='right', overlaying='y', showgrid=False)
)
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=p_dbbm.index, y=p_dbbm - p_bm, mode='lines', name='Diff Price',
                         line=dict(color="black", dash='dot'), yaxis='y'))
fig.update_layout(
    xaxis_title='Date',
    height=200,
    yaxis=dict(title='Price Difference', side='left', showgrid=False),
    yaxis2=dict(side='right', overlaying='y', showgrid=False),
    margin=dict(r=150, t=10),
)
fig.show()

In [ ]:
print("Brownian motion: ", calculate_predictive_corr(p_bm, db_based_signal(p_bm)))
print("Brownian + DB %B:", calculate_predictive_corr(p_dbbm, db_based_signal(p_dbbm)))

# Brownian motion adjusted with the signals combined

In [ ]:
def combined_signal(prices: np.ndarray) -> np.ndarray:
    rsi = ewmrsi_based_signal(prices)
    macd = macd_signal(prices)
    mom = semi_ewm_mom(prices)
    eb = eb_based_signal(prices)
    db = db_based_signal(prices)
    return rsi / np.nanstd(rsi) + macd / np.nanstd(macd) + mom / np.nanstd(mom) + eb / np.nanstd(eb) + db / np.nanstd(db)

comb_model = QuasiBrownianMotion(combined_signal, f_signal_vol=0.2, **params)
p_comb = comb_model.path_sample(**path_params)
signal = combined_signal(p_bm)
colors = np.where(signal < 0, 'red', 'green')

fig = go.Figure()
fig.add_trace(go.Bar(x=p_bm.index, y=signal, marker_color=colors, opacity=0.5, name='Indicator', yaxis='y'))
fig.add_trace(go.Scatter(x=p_bm.index, y=p_bm, mode='lines', name='BM Price', yaxis='y2',
                         line=dict(color=price_color)))
fig.add_trace(go.Scatter(x=p_comb.index, y=p_comb, mode='lines', name='QBM Price', yaxis='y2',
                         line=dict(color=price_color, dash='dot')))
fig.update_layout(
    title='Brownian motion adjusted with combined indicator', xaxis_title='Date', height=600,
    yaxis=dict(title='Indicator', side='left', showgrid=False),
    yaxis2=dict(title='Closing Price', side='right', overlaying='y', showgrid=False)
)
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=p_comb.index, y=p_comb - p_bm, mode='lines', name='Diff Price',
                         line=dict(color="black", dash='dot'), yaxis='y'))
fig.update_layout(
    xaxis_title='Date',
    height=200,
    yaxis=dict(title='Price Difference', side='left', showgrid=False),
    yaxis2=dict(side='right', overlaying='y', showgrid=False),
    margin=dict(r=150, t=10),
)
fig.show()

In [ ]:
print("Brownian motion:    ", calculate_predictive_corr(p_bm, combined_signal(p_bm)))
print("Brownian + combined:", calculate_predictive_corr(p_comb, combined_signal(p_comb)))